In [2]:
import numpy as np
import json
from tensorflow import keras

In [5]:
def get_word_index(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

In [4]:
data = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words=10000) 

In [5]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [8]:
word_index = data.get_word_index() 

word_index = {k:(v+3) for k, v in word_index.items()} 
word_index["<PAD>"] = 0 
word_index["<START>"] = 1 
word_index["<UNK>"] = 2 
word_index["<UNUSED>"] = 3 

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()]) 

In [9]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], padding="post", maxlen=250)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], padding="post", maxlen=250)

In [10]:
def decode_review(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])

In [11]:

print(decode_review(train_data[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [12]:
vocab_size = 10000
embedding_dim = 16
learning_rate = 0.01

In [23]:
def embed(X,E):
    return np.array([E[i] for i in X])

In [13]:
def ReLU(X):
    return np.maximum(0, X)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [14]:
def init_params():
    E = np.random.randn(vocab_size, embedding_dim)
    W1 = np.random.randn(16,16)
    b1 = np.random.randn(16,)
    W2 = np.random.randn(16,1)
    b2 = np.random.randn(1,)
    return E, W1, b1, W2, b2

In [15]:
def forward_prop(E, W1, b1, W2, b2, X):
    Z0 = embed(X,E)
    A1 = Z0
    Z1 = np.mean(A1, axis=0)
    A2 = Z1
    Z2 = A2.dot(W1) + b1
    A3 = ReLU(Z2)
    Z3 = A3.dot(W2) + b2
    A4 = sigmoid(Z3)
    return A1, A2, Z2, A3, Z3, A4

In [16]:
def back_prop(A1, A2, Z2, A3, A4, Y, E, W1, W2, X):
    m=Y.size
    dZ3 = (A4 - Y)/m
    dW2 = A3.reshape(-1, 1) * dZ3
    db2 = dZ3.flatten()

    dA3 = dZ3 * W2.flatten()
    dZ2 = dA3 * (Z2 > 0)
    dW1 = A2.reshape(-1, 1).dot(dZ2.reshape(1, -1))
    db1 = dZ2.flatten()

    dA2 = dZ2.dot(W1.T)
    dZ1 = dA2 / A1.shape[0]
    dA1 = np.tile(dZ1, (A1.shape[0], 1))

    dE = np.zeros_like(E)
    for t, tok in enumerate(X):
        dE[tok] += dA1[t]
    
    return dE, dW1, db1, dW2, db2

In [17]:
def update_params(E, W1, b1, W2, b2, dE, dW1, db1, dW2, db2, learning_rate):
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    E  = E - learning_rate * dE
    return E, W1, b1, W2, b2

In [18]:
def get_predictions(A4):
    return np.argmax(A4, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions==Y)/Y.size

In [ ]:
def gradient_descent(X, Y, iterations, learning_rate):
    E, W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        correct = 0
        for j in range(len(X)):
            A1, A2, Z2, A3, Z3, A4 = forward_prop(E, W1, b1, W2, b2, X[j])
            dE, dW1, db1, dW2, db2 = back_prop(A1, A2, Z2, A3, A4, Y[j], E, W1, W2, X[j])
            E, W1, b1, W2, b2 = update_params(E, W1, b1, W2, b2, dE, dW1, db1, dW2, db2, learning_rate)

            prediction = 1 if A4[0] > 0.5 else 0
            if prediction == Y[j]:
                correct += 1
                
        if i % 10 == 0:
            accuracy = correct / len(X)
            print("Iteration:", i, "Accuracy: ", accuracy)
    return E, W1, b1, W2, b2

In [24]:

E, W1, b1, W2, b2 = gradient_descent(train_data, train_labels, 100, learning_rate)

Iteration: 0 Accuracy:  0.56052
Iteration: 10 Accuracy:  0.75048
Iteration: 10 Accuracy:  0.75048
Iteration: 20 Accuracy:  0.7992
Iteration: 20 Accuracy:  0.7992
Iteration: 30 Accuracy:  0.82392
Iteration: 30 Accuracy:  0.82392
Iteration: 40 Accuracy:  0.8396
Iteration: 40 Accuracy:  0.8396
Iteration: 50 Accuracy:  0.85136
Iteration: 50 Accuracy:  0.85136
Iteration: 60 Accuracy:  0.85968
Iteration: 60 Accuracy:  0.85968
Iteration: 70 Accuracy:  0.86908
Iteration: 70 Accuracy:  0.86908
Iteration: 80 Accuracy:  0.87464
Iteration: 80 Accuracy:  0.87464
Iteration: 90 Accuracy:  0.8812
Iteration: 90 Accuracy:  0.8812


In [227]:
def test_model(E, W1, b1, W2, b2, X_test, Y_test):
    correct = 0
    for i in range(len(X_test)):
        A1, A2, Z2, A3, Z3, A4 = forward_prop(E, W1, b1, W2, b2, X_test[i])
        prediction = 1 if A4[0] > 0.5 else 0
        if prediction == Y_test[i]:
            correct += 1
        if prediction == 0:
            print(A4[0])
            print(i)
            break
    
    accuracy = correct / len(X_test)
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy

In [228]:
test_model(E, W1, b1, W2, b2, test_data,test_labels)

0.3082849029847548
7
Test Accuracy: 0.0002


0.00024

In [190]:
def testing(E, W1, b1, W2, b2, test_text, i):
    A1, A2, Z2, A3, Z3, A4 = forward_prop(E, W1, b1, W2, b2, test_data[i])
    prediction = 1 if A4[0] > 0.5 else 0
    if(prediction==1):
        print("positive")
    else:
        print("negative")    
    print(A4[0])

In [169]:
def text_to_ids(sentence, word_index, unk_id=0):
    words = sentence.lower().split()
    ids = [word_index.get(w, unk_id) for w in words]
    return ids
p_sentence = "focuses on mood and character development the plot is very simple and many of the scenes take place on the same set in frances <UNK> the sandy dennis character apartment but the film builds to a disturbing climax br br the characters create an atmosphere <UNK> with sexual tension and psychological <UNK> it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of her darker characters but she plays it so perfectly and convincingly that it's scary michael burns does a good job as the mute young man regular altman player michael murphy has a small part the <UNK> moody set fits the content of the story very well in short this movie is a powerful study of loneliness sexual <UNK> and desperation be patient <UNK> up the atmosphere and pay attention to the wonderfully written script br br i praise robert altman this is one of his many films that deals with unconventional fascinating subject matter this film is disturbing but it's sincere and it's sure to <UNK> a strong emotional response from the viewer if you want to see an unusual film some might even say bizarre this is worth the time br br unfortunately it's very difficult to find in video stores you may have to buy it off the internet"#"This movie straight-up crawled under my skin in the softest way possible, like a ghost giving me a weighted blanket hug 😭✨ The whole vibe felt dreamlike but not in that confusing “what is happening” way—more like drifting through a memory you forgot you loved. Every frame looked handcrafted, like the director dipped their fingers into the screen and rearranged reality just for fun. The acting? Bro… unreal. The lead carried this quiet storm energy that made every tiny expression hit like a prophecy. Even when nobody was talking, the silence felt alive, slithering around the edges like it knew secrets it refused to share. The score wrapped around the story like a heartbeat—steady, comforting, but always a little off-kilter, reminding you something bigger was coming. And when the emotional punch finally landed in the final act? I swear my soul did a somersault. It’s the kind of film that follows you home, sits in your room, and whispers, “think about me.” And I do. I will. Forever. 10/10, honestly felt like being gently haunted by beauty."
n_sentence = "<START> please give this one a miss br br <UNK> <UNK> and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite <UNK> so all you madison fans give this a miss <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"#"Man… this movie cooked me emotionally but not in a good way 💀 Like, it tried so desperately to feel profound that it ended up chasing its own shadow for two hours. Every scene dragged itself across the floor, begging to matter, but never getting there. The dialogue felt like someone fed an AI a motivational poster and said “make it deep.” Characters wandered around like NPCs who forgot their quest lines. Nothing they said or did connected to anything. The story kept hinting at meaning, but every reveal landed flatter than a broken jump scare. I swear the plot twists were so predictable I called one while half-asleep scrolling my phone. The soundtrack was dramatic for no reason—violins screaming while someone boiled pasta or stared at a lamp like it held the secrets of the universe. Bro, I was fighting for my life trying not to laugh. By the end, I felt like the movie siphoned my energy, my patience, and maybe part of my lifespan. 2/10, only because the credits rolled and freed me."
p_array = text_to_ids(p_sentence, word_index)
n_array = text_to_ids(n_sentence, word_index)
print(p_array) 
print(n_array) 

[2679, 23, 1310, 5, 109, 943, 4, 114, 9, 55, 606, 5, 111, 7, 4, 139, 193, 273, 23, 4, 172, 270, 11, 7216, 0, 4, 8463, 2801, 109, 1603, 21, 4, 22, 3861, 8, 6, 1193, 1330, 10, 10, 4, 105, 987, 35, 841, 0, 19, 861, 1074, 5, 1987, 0, 45, 55, 221, 15, 670, 5304, 526, 14, 1069, 4, 405, 5, 2438, 7, 27, 85, 108, 131, 4, 5045, 5304, 3884, 405, 9, 3523, 133, 5, 50, 13, 104, 51, 66, 166, 14, 22, 157, 9, 4, 530, 239, 34, 8463, 2801, 45, 407, 31, 7, 41, 3778, 105, 21, 59, 299, 12, 38, 950, 5, 4521, 15, 45, 629, 488, 2733, 127, 6, 52, 292, 17, 4, 6936, 185, 132, 1988, 5304, 1799, 488, 2693, 47, 6, 392, 173, 4, 0, 4378, 270, 2352, 4, 1500, 7, 4, 65, 55, 73, 11, 346, 14, 20, 9, 6, 976, 2078, 7, 5293, 861, 0, 5, 4182, 30, 3127, 0, 56, 4, 841, 5, 990, 692, 8, 4, 1669, 398, 229, 10, 10, 13, 2822, 670, 5304, 14, 9, 31, 7, 27, 111, 108, 15, 2033, 19, 7836, 1429, 875, 551, 14, 22, 9, 1193, 21, 45, 4829, 5, 45, 252, 8, 0, 6, 565, 921, 3639, 39, 4, 529, 48, 25, 181, 8, 67, 35, 1732, 22, 49, 238, 60, 135, 1162

In [260]:
test_text=[
    [1,13,784,14,20, 2 ,12,9,38,357],
    [1,13,66,423,12],
    [1,13,1539,18,164, 2 ,87],
    [1,118, 20, 126],
    [1, 591, 202, 14, 31, 6, 717, 10, 10, 2, 2, 5, 4, 360, 7, 4, 177, 5760, 394, 354, 4, 123, 9, 1035, 1035, 1035, 10, 10, 13, 92, 124, 89, 488, 7944, 100, 28, 1668, 14, 31, 23, 27, 7479, 29, 220, 468, 8, 124, 14, 286, 170, 8, 157, 46, 5, 27, 239, 16, 179, 0, 38, 32, 25, 7944, 451, 202, 14, 6, 717, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [2679, 23, 1310, 5, 109, 943, 4, 114, 9, 55, 606, 5, 111, 7, 4, 139, 193, 273, 23, 4, 172, 270, 11, 7216, 2, 4, 8463, 2801, 109, 1603, 21, 4, 22, 3861, 8, 6, 1193, 1330, 10, 10, 4, 105, 987, 35, 841, 2, 19, 861, 1074, 5, 1987, 2, 45, 55, 221, 15, 670, 5304, 526, 14, 1069, 4, 405, 5, 2438, 7, 27, 85, 108, 131, 4, 5045, 5304, 3884, 405, 9, 3523, 133, 5, 50, 13, 104, 51, 66, 166, 14, 22, 157, 9, 4, 530, 239, 34, 8463, 2801, 45, 407, 31, 7, 41, 3778, 105, 21, 59, 299, 12, 38, 950, 5, 4521, 15, 45, 629, 488, 2733, 127, 6, 52, 292, 17, 4, 6936, 185, 132, 1988, 5304, 1799, 488, 2693, 47, 6, 392, 173, 4, 2, 4378, 270, 2352, 4, 1500, 7, 4, 65, 55, 73, 11, 346, 14, 20, 9, 6, 976, 2078, 7, 5293, 861, 2, 5, 4182, 30, 3127, 0, 56, 4, 841, 5, 990, 692, 8, 4, 1669, 398, 229, 10, 10, 13, 2822, 670, 5304, 14, 9, 31, 7, 27, 111, 108, 15, 2033, 19, 7836, 1429, 875, 551, 14, 22, 9, 1193, 21, 45, 4829, 5, 45, 252, 8, 2, 6, 565, 921, 3639, 39, 4, 529, 48, 25, 181, 8, 67, 35, 1732, 22, 49, 238, 60, 135, 1162, 14, 9, 290, 4, 58, 10, 10, 472, 45, 55, 878, 8, 169, 11, 374, 5687, 25, 203, 28, 8, 818, 12, 125, 4, 3077],
    [   1,    4,    2,  745,    2,  912,    9,    2,    8,    2, 1820, 5889,  912,  190,   54,    2, 8408, 7024, 2957,   11,  513,    8, 2225,    6, 1853, 1799,    8,    4, 5110,   27,  912,    9, 2982,   34,    2,  912,  103,    6, 1060, 1274,  121,    2,    9,    2,   39,  513,    6,    2, 5692,   15,    2,  912,  215, 6275,  912,   38,   15,   59,   70, 3083,   41,    2, 3564,   14,    9, 6749,    5,    4, 2515,  809,   11,  119,   38,   81,    2,    5,    2,    4,  360,    7,    4,   22, 4097,   34,   19,  883,    5,   33,    4,  130,    2,  912,  408, 2540,   21,   37,    9,    4,  336,   10,   10,    4,  912,   65,    9,    4,   55,  815, 2196,   15,    9,  343,    8,  353,    5,  987,    6,   65,  200, 1714,   94,    6,  394,  769,   50,   26,  342,  293,  621, 1328,   32,    7,   63,   26, 1916,   78,  690,    5, 2150, 3822,   94,   43,   35,  576,  357,   22, 8408,   47,   99,  111,  715,   11,  257, 4130,    5, 1545,   98,   11,   35,  220, 2227, 1377,   94,   24,  163,  126,   21,   94,  981,    8,   30, 8408,    5, 1820,   28,  224,   76,  128,   74,   14,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]]
#test_text=np.array([a + [0]*(250 - len(a)) for a in test_text])
y_test = [0, 1, 0, 1, 0, 1]
#print(test_text[4])
#print(decode_review(test_text[2]))

In [261]:
testing(E, W1, b1, W2, b2, test_text, 6)
print(np.array(test_text[6]))
print(test_data[7])
#test_model(E, W1, b1, W2, b2, test_text, y_test)

positive
0.5879252373516386
[   1    4    2  745    2  912    9    2    8    2 1820 5889  912  190
   54    2 8408 7024 2957   11  513    8 2225    6 1853 1799    8    4
 5110   27  912    9 2982   34    2  912  103    6 1060 1274  121    2
    9    2   39  513    6    2 5692   15    2  912  215 6275  912   38
   15   59   70 3083   41    2 3564   14    9 6749    5    4 2515  809
   11  119   38   81    2    5    2    4  360    7    4   22 4097   34
   19  883    5   33    4  130    2  912  408 2540   21   37    9    4
  336   10   10    4  912   65    9    4   55  815 2196   15    9  343
    8  353    5  987    6   65  200 1714   94    6  394  769   50   26
  342  293  621 1328   32    7   63   26 1916   78  690    5 2150 3822
   94   43   35  576  357   22 8408   47   99  111  715   11  257 4130
    5 1545   98   11   35  220 2227 1377   94   24  163  126   21   94
  981    8   30 8408    5 1820   28  224   76  128   74   14    0    0
    0    0    0    0    0    0    0    0    0    

In [220]:
print(decode_review(test_text[6]))

<START> the <UNK> richard <UNK> dog is <UNK> to <UNK> joan fontaine dog however when <UNK> bing crosby arrives in town to sell a record player to the emperor his dog is attacked by <UNK> dog after a revenge attack where <UNK> is <UNK> from town a <UNK> insists that <UNK> dog must confront dog so that she can overcome her <UNK> fears this is arranged and the dogs fall in love so do <UNK> and <UNK> the rest of the film passes by with romance and at the end <UNK> dog gives birth but who is the father br br the dog story is the very weak vehicle that is used to try and create a story between humans its a terrible storyline there are 3 main musical pieces all of which are rubbish bad songs and dreadful choreography its just an extremely boring film bing has too many words in each sentence and delivers them in an almost irritating manner its not funny ever but its meant to be bing and joan have done much better than this <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

In [232]:
i=7
print(decode_review(test_data[i]))
print(test_labels[i])

<START> the <UNK> richard <UNK> dog is <UNK> to <UNK> joan fontaine dog however when <UNK> bing crosby arrives in town to sell a record player to the emperor his dog is attacked by <UNK> dog after a revenge attack where <UNK> is <UNK> from town a <UNK> insists that <UNK> dog must confront dog so that she can overcome her <UNK> fears this is arranged and the dogs fall in love so do <UNK> and <UNK> the rest of the film passes by with romance and at the end <UNK> dog gives birth but who is the father br br the dog story is the very weak vehicle that is used to try and create a story between humans its a terrible storyline there are 3 main musical pieces all of which are rubbish bad songs and dreadful choreography its just an extremely boring film bing has too many words in each sentence and delivers them in an almost irritating manner its not funny ever but its meant to be bing and joan have done much better than this <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

In [207]:
print("E: ", E)
print("W1: ", W1)
print("b1: ", b1)
print("W2: ", W2)
print("b2: ", b2)

E:  [[ 0.07509268 -0.21391617 -0.03601754 ... -0.01147904  0.49341896
  -0.00853286]
 [-0.2465896  -0.12930816  0.50363968 ... -0.60155447 -2.27323984
  -0.5965225 ]
 [ 0.9155115   0.6678291   0.51701345 ... -0.27287972  0.52690106
  -1.19417843]
 ...
 [ 0.03890993 -0.14127242 -1.6207738  ... -0.24703842 -2.0588844
  -0.55662718]
 [ 2.17807355  0.82876638  0.74720959 ... -1.2772134   1.30240427
  -0.71623748]
 [ 0.15765434 -0.38588308  0.39095317 ...  1.48876608  0.59307307
  -0.07973281]]
W1:  [[-3.82990183e-01 -1.15133608e+00  2.71973946e+00 -1.48713060e+00
  -1.03335065e+00 -1.54211966e+00 -3.87914614e-01  1.15709145e+00
  -2.25065353e+00 -1.04185128e+00 -1.45163154e+00  1.53575655e+00
   1.59395131e-02 -1.92718732e-01  1.96111696e+00 -2.62865262e-01]
 [-8.93844599e-01 -1.79663788e+00 -2.28197916e+00  1.03414458e+00
  -8.92284970e-05  9.74888135e-02  6.41392069e-01  8.25961700e-01
   2.26807922e+00  4.95560924e-01  1.66579509e-02  1.14290000e+00
   7.86936810e-01  6.98335208e-01 -2.

In [262]:
np.savez("my_model.npz",
         E=E,
         W1=W1,
         b1=b1,
         W2=W2,
         b2=b2)